In [24]:
import numpy as np
import pandas as pd
import scipy.sparse
from scipy.spatial.distance import correlation

In [25]:
data=pd.read_csv('data_collaborative.csv')
placeInfo=pd.read_csv('data_content.csv')

In [26]:
data.head()

,userId,itemId,rating,timestamp
0,1,1,NaN,881250949
1,1,2,3.0,891717742
2,1,3,1.0,878887116
3,1,4,NaN,880606923
4,1,5,1.0,886397596


In [27]:
placeInfo.head()

,category,distance,duration,nearby_places,title,url,p_rating,count,itemId
0,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158,1
1,Heritage,6 Kms,1-2 Hours,None,Nahargarh Fort,/rajasthan/jaipur/nahargarh-fort,4.6,652,2
2,Heritage,13 Kms,2-3 Hours,"Jaigarh Fort (1 km by walk), Srijagat Siromani...",Amer Fort / Amber Fort,/rajasthan/jaipur/amer-fort-amber-fort,5.0,783,3
3,Pilgrimage,6 Kms,1-2 Hours,None,Birla Mandir,/rajasthan/jaipur/birla-mandir,4.9,482,4
4,Heritage,6 Kms,30 Mins,None,Hawa Mahal,/rajasthan/jaipur/hawa-mahal,4.2,890,5


In [28]:
data=pd.merge(data, placeInfo, left_on='itemId', right_on="itemId")
userIds=data.userId
userIds2=data[['userId']]
data.head()

,userId,itemId,rating,timestamp,category,distance,duration,nearby_places,title,url,p_rating,count
0,1,1,NaN,881250949,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
1,2,1,NaN,879539727,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
2,3,1,4.0,884920949,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
3,4,1,4.0,879373848,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
4,5,1,3.0,878089182,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158


In [31]:
data.loc[0:10,['userId']]
data=pd.DataFrame.sort_values(data, ['userId','itemId'], ascending=[0,1])

In [32]:
data.head()

,userId,itemId,rating,timestamp,category,distance,duration,nearby_places,title,url,p_rating,count
8,9,1,NaN,878089716,Wildlife,6 Kms,1-2 Hours,None,Jaipur Zoo,/rajasthan/jaipur/jaipur-zoo,2.2,158
17,9,2,3.0,890881335,Heritage,6 Kms,1-2 Hours,None,Nahargarh Fort,/rajasthan/jaipur/nahargarh-fort,4.6,652
26,9,3,3.0,889502324,Heritage,13 Kms,2-3 Hours,"Jaigarh Fort (1 km by walk), Srijagat Siromani...",Amer Fort / Amber Fort,/rajasthan/jaipur/amer-fort-amber-fort,5.0,783
35,9,4,3.0,879525876,Pilgrimage,6 Kms,1-2 Hours,None,Birla Mandir,/rajasthan/jaipur/birla-mandir,4.9,482
44,9,5,NaN,879485532,Heritage,6 Kms,30 Mins,None,Hawa Mahal,/rajasthan/jaipur/hawa-mahal,4.2,890


In [33]:
def favoritePlace(activeUser,N):
    print("favoritePlace")
    topPlace=pd.DataFrame.sort_values(
        data[data.userId==activeUser],['rating'],ascending=[0])[:N]
    return list(topPlace.title)

In [34]:
userItemRatingMatrix=pd.pivot_table(data, values='rating',index=['userId'], columns=['itemId'])

In [35]:
userItemRatingMatrix

itemId,1,2,3,4,5,6,7,8,9,10,...,23,24,25,26,27,28,29,30,31,32
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,3.0,1.0,NaN,1.0,NaN,NaN,5.0,3.0,NaN,...,NaN,2.0,4.0,NaN,5.0,NaN,4.0,NaN,3.0,NaN
2,NaN,4.0,NaN,4.0,1.0,NaN,1.0,4.0,NaN,4.0,...,5.0,NaN,NaN,5.0,5.0,NaN,3.0,NaN,4.0,3.0
3,4.0,5.0,NaN,5.0,NaN,4.0,NaN,3.0,1.0,3.0,...,5.0,3.0,NaN,4.0,NaN,4.0,4.0,NaN,2.0,NaN
4,4.0,NaN,4.0,NaN,NaN,5.0,4.0,3.0,NaN,NaN,...,NaN,4.0,4.0,NaN,4.0,5.0,NaN,2.0,5.0,NaN
5,3.0,3.0,NaN,3.0,4.0,NaN,NaN,3.0,5.0,5.0,...,4.0,3.0,3.0,NaN,3.0,4.0,4.0,NaN,4.0,4.0
6,NaN,NaN,5.0,3.0,NaN,2.0,3.0,NaN,4.0,4.0,...,3.0,2.0,NaN,4.0,3.0,NaN,4.0,5.0,NaN,5.0
7,3.0,NaN,4.0,NaN,3.0,3.0,2.0,NaN,3.0,4.0,...,4.0,4.0,3.0,NaN,NaN,2.0,5.0,NaN,4.0,4.0
8,NaN,3.0,NaN,1.0,4.0,5.0,NaN,5.0,3.0,NaN,...,2.0,4.0,NaN,NaN,4.0,5.0,NaN,3.0,3.0,NaN
9,NaN,3.0,3.0,3.0,NaN,5.0,5.0,NaN,2.0,4.0,...,NaN,NaN,4.0,1.0,1.0,NaN,3.0,NaN,4.0,NaN


In [36]:
def similarity(user1,user2):
    print("similarity")
    try:
        user1=np.array(user1)-np.nanmean(user1)
        user2=np.array(user2)-np.nanmean(user2)
        commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
        if len(commonItemIds)==0:
           return 0
        else:
           user1=np.array([user1[i] for i in commonItemIds])
           user2=np.array([user2[i] for i in commonItemIds])
           return correlation(user1,user2)
    except ZeroDivisionError:
        print("You can't divide by zero!")

In [40]:
def nearestNeighbourRatings(activeUser,K):
    print("NearestNeighbours")
    try:
        similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])
        similarityMatrix.head()        
        for i in userItemRatingMatrix.index:
            similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])
        similarityMatrix=pd.DataFrame.sort_values(similarityMatrix,['Similarity'],ascending=[0])
        nearestNeighbours=similarityMatrix[:K]
        neighbourItemRatings=userItemRatingMatrix.loc[nearestNeighbours.index]
        predictItemRating=pd.DataFrame(index=userItemRatingMatrix.columns, columns=['Rating'])
        for i in userItemRatingMatrix.columns:
            predictedRating=np.nanmean(userItemRatingMatrix.loc[activeUser])
            for j in neighbourItemRatings.index:
                if userItemRatingMatrix.loc[j,i]>0:
                   predictedRating += (userItemRatingMatrix.loc[j,i]-np.nanmean(userItemRatingMatrix.loc[j]))*nearestNeighbours.loc[j,'Similarity']
                predictItemRating.loc[i,'Rating']=predictedRating
    except ZeroDivisionError:
        print("You can't divide by zero!")            
    return predictItemRating

In [41]:
def topNRecommendations(activeUser,N):
    print("topNRecommendation")
    try:
        predictItemRating=nearestNeighbourRatings(activeUser,10)
        placeAlreadyWatched=list(userItemRatingMatrix.loc[activeUser].loc[userItemRatingMatrix.loc[activeUser]>0].index)
        predictItemRating=predictItemRating.drop(placeAlreadyWatched)
        topRecommendations=pd.DataFrame.sort_values(predictItemRating, ['Rating'],ascending=[0])[:N]
        topRecommendationTitles=(placeInfo.loc[placeInfo.itemId.isin(topRecommendations.index)])
    except ZeroDivisionError:
        print("You can't divide by zero!")
    return list(topRecommendationTitles.title)

In [42]:
activeUser=int(input("Enter userid: "))
print("The user's favorite places are: ")
print(favoritePlace(activeUser,5))
print("The recommended places for you are: ")
print(topNRecommendations(activeUser,4))

Enter userid: 2
The user's favorite places are: 
favoritePlace
['Jama Masjid / Akbari Mosque - Amer', 'Srijagat Siromaniji Temple', 'Sargasuli Tower / Isar Lat', 'Statue Circle / Sawai Jai Singh Circle', 'Gatore ki Chhatriyan']
The recommended places for you are: 
topNRecommendation
NearestNeighbours
similarity
similarity
similarity
similarity
similarity
similarity
similarity
similarity
similarity
['Jaipur Zoo', 'Amer Fort / Amber Fort', 'Maharani Ki Chhatri', 'City Palace / Sawai Man Singh II Museum']


In [44]:
similarityMatrix=pd.DataFrame(index=userItemRatingMatrix.index,columns=['Similarity'])
similarityMatrix

,Similarity
userId,
1,NaN
2,NaN
3,NaN
4,NaN
5,NaN
6,NaN
7,NaN
8,NaN
9,NaN


In [45]:
for i in userItemRatingMatrix.index:
    similarityMatrix.loc[i]=similarity(userItemRatingMatrix.loc[activeUser],userItemRatingMatrix.loc[i])

similarity
similarity
similarity
similarity
similarity
similarity
similarity
similarity
similarity


C:\ProgramData\Anaconda3\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [54]:
user1=np.array(user1)-np.nanmean(user1)
user2=np.array(user2)-np.nanmean(user2)
commonItemIds=[i for i in range(len(user1)) if user1[i]>0 and user2[i]>0]
user1=np.array([user1[i] for i in commonItemIds])
user2=np.array([user2[i] for i in commonItemIds])

In [55]:
user1

array([1.88888889, 1.88888889, 0.88888889, 1.88888889])

In [52]:
user1 = userItemRatingMatrix.loc[1]
user2 = userItemRatingMatrix.loc[2]

In [56]:
user2

array([0.3, 1.3, 1.3, 1.3])